# An Analytic Example of Bayesian Quadrature Optimization

## 0. Imports

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import jax.numpy as jnp
import random
from bqo_acquisition_for_wos.sbq import SBQ
from bqo_acquisition_for_wos.kernel import rbf
from functools import partial
from scipy.stats import norm
import matplotlib.pyplot as plt
from emukit.examples.vanilla_bayesian_quadrature_with_rbf.vanilla_bq_loop_with_rbf import create_vanilla_bq_loop_with_rbf_kernel
from emukit.quadrature.measures.gaussian_measure import GaussianMeasure

## 1. Example Set Up
In this example, we present an example problem in the Bayesian Quadrature Optimization setting with a simple analytic test function. Specifically, for $F(\theta, x) = -\theta^2 + x$, we want to solve the optimization problem $$\max_{[-1/2, 1/2]} \int_{x} F(\theta, x)p(x) dx$$ where $p(x)$ is the density of a standard Gaussian random variable. 

In [3]:
# Define the analytic function that we are going to set up a basic experiment with baselines with
def F(theta,x):
    return -theta ** 2 + x

# Define the ground truth of the integral of f w.r.t probability distribution p(x)
def G(theta):
    return -theta ** 2

# The range of x points
THETA_RANGE = [-1/2, 1/2]

## 2. Random Search Baseline

First, we will implement a basic random search baseline for this problem. This involves, for a fixed number of iterations $N$, the following:
1. Choose a random parameter $\theta\in[-1/2,1/2]$ uniformly
2. Estimate $\int_x F(\theta, x)p(x)dx$ with Bayesian Quadrature (run $T$ posterior updates for some fixed $T$)
3. Keep track of the mean of the current posterior distribution

At the end of the $N$ iterations, we will return the last (current) estimate. To keep computational time for this example relatively reasonable, we choose a $T$ and $N$ accordingly.

In [ ]:
# How many steps to run BQ and how many times to run random search loop
T = 20
N = 10

# Constants for plotting
LEGEND_SIZE = 15
FIGURE_SIZE = (12, 8)

In [ ]:
# Initialize current BQ loop
bq_loop = None
for _ in range(N):
    # Log every 10 iterations
    if (_ + 1) % 10 == 0:
        print(f"Iteration: {_ + 1}/{N}")

    # Initialize the starting data for underlying GP model (starting x points are fixed)
    X_init = np.array([[-0.5], [0.1], [0.4]])
    theta = random.uniform(*THETA_RANGE)
    Y_init = np.array([F(theta, x) for x in X_init]).reshape(-1, 1)

    # Create the BQ loop with RBF kernel
    bq_loop = create_vanilla_bq_loop_with_rbf_kernel(X_init, Y_init, measure=GaussianMeasure(mean=np.array([0]), variance=1.0))
    user_f = partial(F, theta)

    # Run the BQ loop
    bq_loop.run_loop(user_f, T)

# Get the final posterior mean as the esimate
x_mesh = np.linspace(*THETA_RANGE, 300).reshape(-1,1)
estimate = bq_loop.model.integrate()[0]

Iteration 1/10
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10


In [27]:
# Compare with ground truth and print results
true_max = 0
print("True max: ", true_max)
print("Estimate: ", estimate)
print("Error: ", abs(true_max - estimate))

True max:  0
Estimate:  -0.16150841968891633
Error:  0.16150841968891633


## 3. Random Search Variant Baseline

Next, we will implement a random search variant in the spirit of Gittins Index based experiments. First, we will divide the interval $[-1/2, 1/2]$ into a mesh of some $n$ number of equally spaced points. We will create a `spaces` array of length $n$ for which `spaces[i]` represents the number of times point $i$ has been visited. Then, for $N$ iterations and a fixed $T$ representing the length of each SBQ procedure, we will do the following:
1. Choose a random integer $i$ from $0$ to $n$
2. If `spaces[i] != T`: 
    - Get the function evaluation $f(\theta_i, x_{spaces[i]})$ (the sequence of $x$ points are fixed by the initial points in BQ model)
    - Increment `spaces[i]`
    - If `spaces[i] == T`:
        - Find the posterior mean of the BQ model with the function evaluations and the given $x$ points, and if this posterior mean is larger than the running max, set it as the max estimate
3. Return the max posterior mean

In [ ]:
# Set constants for this procedure
n_mesh = 50
N = 500
T = 20

In [ ]:
# Set inital X points and set up the mesh in theta-space of n_mesh points, and initialize the "spaces" and function evals arrays
X_init = jnp.array([[-0.5], [0.1], [0.4]])
theta_mesh = jnp.linspace(*THETA_RANGE, n_mesh)
spaces = [0] * n_mesh
function_evals = jnp.zeros((n_mesh, T + 3))

# Fill the first three columns of function_evals with evaluations of the function F at the initial X points
function_evals[:, :3] = jnp.array([[F(theta, x) for x in X_init] for theta in theta_mesh]).T

# Create the SBQ object and determine the order of x points from the SBQ procedure
sbq = SBQ(rbf, norm, [[-1000], [1000]], X_init)
sbq.run_sbq_procedure(n_steps=T)

# Keep track of maximum posterior mean across the theta space (our mesh)
max_posterior_mean = 0

# Run the iterations of the random search variant
for _ in range(N):
    # Log every 10 iterations
    if (_ + 1) % 10 == 0:
        print(f"Iteration: {_ + 1}/{N}")

    # Get a random theta from the mesh
    i = random.randint(0, n_mesh - 1)
    theta = theta_mesh[i]

    # If the theta has not been sampled T times, reveal a function evaluation
    if spaces[i] < T:
        function_evals[i][spaces[i]] = F(theta, X_init[spaces[i]])
        spaces[i] += 1

        # If we have sampled T times, we can compute the BQ estimate
        if spaces[i] == T:
            sbq.reset_Y()
            sbq.add_Y(function_evals[i])
            max_posterior_mean = max(max_posterior_mean, sbq.posterior_mean())

# Print the maximum posterior mean found
print("Maximum posterior mean found: ", max_posterior_mean)

Plot: Regret (mean deviation from ground truth + standard error at each time step in the loop vs. loop iterations/time steps)

## 3. Thompson Sampling Baseline
For the final baseline, we will employ a Thompson sampling style baseline. In the last procedure, when obtaining a function evaluation, instead of exact function evaluations, we will sample a path from the GP posterior conditioned on all the $x$ points and $y$ values seen up to that point. Specifically, we will divide $[-1/2, 1/2]$ into a mesh of $n$ points again, have `spaces` and $T$ signify the same variables as before, and do the following for $N$ iterations:
1. Choose a random integer $i$ from $0$ to $n$
2. If `spaces[i] != T`: 
    - Get the function evaluation as the $y$-value along a sample path from $GP(\mathbf{y}\mid x_0, \dots x_{spaces[i]-1}, y_0, \dots y_{spaces[i]-1})$ at $x_{spaces[i]}$
    - Increment `spaces[i]`
    - If `spaces[i] == T`:
        - Find the posterior mean of the BQ model with the function evaluations and the given $x$ points, and if this posterior mean is larger than the running max, set it as the max estimate
3. Return the max posterior mean

In [4]:
# Test for the SBQ class
theta = -0.1
X_init = jnp.array([[-0.5], [0.1], [0.4]])
sbq = SBQ(rbf, norm, [[-1000], [1000]], X_init)
sbq.run_sbq_procedure(n_steps=T)

# Add the Y values for the given theta
Y_values = jnp.array([F(theta, x) for x in X_init]).reshape(-1, 1)
sbq.add_Y(Y_values)

# Get the posterior mean for the given theta
posterior_mean = sbq.posterior_mean(theta)
print(f"Posterior mean for theta={theta}: {posterior_mean}")

ValueError: not enough values to unpack (expected 2, got 1)